# Igraph clusters

In [1]:
import igraph as ig
import leidenalg as la
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from itertools import chain

Data from spectral_clustering data cleaning process

In [2]:
from get_graph_from_data import get_graph, draw  
G_di, A_di, G_un, A_un = get_graph(data=pd.read_csv('../data/chicago_taxi_trips_2016_11.csv'), head=False) 
# draw(G_di) 
# draw(G_un) 

In [3]:
#covert to igraph object
G_un_ig = ig.Graph.from_networkx(G_un)


Leiden clusters

In [4]:
partition = la.find_partition(G_un_ig, la.ModularityVertexPartition)

In [7]:
partition2 = la.find_partition(G_un_ig, la.CPMVertexPartition,
                                   resolution_parameter = 0.05)

In [67]:
#init list of values for gamma, node number, and cluster membership
gammas = np.arange(0.02, 0.2, 0.02)
g_ls = [] #value for gamma
cl_num_ls = [] #number of clusters from this gamma
nodes = [] #node numbers
node_mems = [] #cluster memberships

#try different values for gamma to get clusters of different sizes
for i in gammas:
    partition = la.find_partition(G_un_ig, la.CPMVertexPartition,
                                   resolution_parameter = i)
    
    n_nodes = len(partition.membership)
    g_ls.append([i]*n_nodes)
    
    
    # Find number of clusters
    n_clust = len(set(partition.membership))
    cl_num_ls.append([n_clust]*n_nodes)

    # Retrieve cluster membership of each node
    node = [*range(len(partition.membership))] #number each node
    node_mem = partition.membership
    
    nodes.append(node)
    node_mems.append(node_mem)


#create ndarray of value for gamma, cluster num, node number, and node membership  
node_mem_gamma = np.asarray(np.column_stack((list(chain(*g_ls)),
                                  list(chain(*cl_num_ls)),
                                  list(chain(*nodes)),
                                  list(chain(*node_mems)))))
#print(node_mem_gamma[:10])

[[0.02 1.   0.   0.  ]
 [0.02 1.   1.   0.  ]
 [0.02 1.   2.   0.  ]
 [0.02 1.   3.   0.  ]
 [0.02 1.   4.   0.  ]
 [0.02 1.   5.   0.  ]
 [0.02 1.   6.   0.  ]
 [0.02 1.   7.   0.  ]
 [0.02 1.   8.   0.  ]
 [0.02 1.   9.   0.  ]]


Community Walktrap (based on random walks)

In [11]:
wc = G_un_ig.community_walktrap()

wc_cl = wc.as_clustering() #creates an object of type VertexClustering, with optimal number of clusters
print(wc_cl.summary())

node_mem = wc_cl.membership #gives cluster membership for each node
node_num = [*range(77)] #number each node

wc_node_mem = np.asarray(np.column_stack((node_num, node_mem)))

Clustering with 77 elements and 3 clusters


In [13]:
#print(wc_node_mem)

In [16]:
draw(wc_cl)

ValueError: too many values to unpack (expected 2)